In [ ]:
# Import Libraries
import json
import os
import tiktoken
import time
from openai import AzureOpenAI

In [ ]:
# Set the environment variables
os.environ["AZURE_OPENAI_API_KEY"] = "REPLACE_WITH_YOUR_KEY_VALUE"
os.environ["AZURE_OPENAI_ENDPOINT"] = "REPLACE_WITH_YOUR_ENDPOINT"

In [ ]:
# Create a client object
client = AzureOpenAI(
 azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
 api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
 api_version="2024-02-01"
)

In [ ]:
# Set your model deployments
deployment_4 = "your-gpt-4-model"
deployment_3_turbo = "your-gpt-3.5-turbo-model"

In [ ]:
# Define messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Can I create a chatbot using Azure OpenAI to chat with my own data?"},
    {"role": "assistant", "content": "Yes, you can use Azure OpenAI and Azure cognitive search to create a chatbot."},
    {"role": "user", "content": "Can you provide more information about Azure cognitive search?"}
]

In [ ]:
# Create System and User Messages
system_message = """You are a translation assistant. A user will provide you with a text to translate between triple pluses
(+++), a target language between triple stars (***), and context 
around the translation between triple ampersands (&&&)."""

user_message = """+++Dear customer, we regret to inform that the 
item you ordered is currently-out-of-stock.+++ ***Spanish*** 
&&&We are trying to send detailed yet polite e-mails to 
customers.&&&"""

In [ ]:
# Run chat
results = client.chat.completions.create(
 model=deployment_4,
 messages=messages)

In [ ]:
# Obtain Results
results_text = results.choices[0].message.content
print(results_text)

In [ ]:
# There's more... System Message
system_message = """You are multi-language translation 
assistant. A user will provide you with a text to translate 
between triple pluses (+++) and context around the translation 
between triple ampersands (&&&). You will translate the text 
into Spanish, French, German, and Japanese. Once you do that, 
you will output the translation in JSON format.
To accomplish this task, first read the translation context, then 
translate the text into Spanish, French, German, and Japanese in 
that order, then output the following JSON format:
{
 "Spanish": "",
 "French": "",
 "German": "", 
 "Japanese": "" 
}
Only output the JSON and ensure that it is the correct 
format."""

# There's more... User Message
user_message = """+++Henry, Tomorrow I will be late for work due 
to unforeseen circumstances. My apologies, Mark.+++ &&&This is 
an e-mail from a worker to his boss.&&&"""

# Set Messages
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

In [ ]:
# Function to get token count for OpenAI
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    enc = tiktoken.encoding_for_model("gpt-4")
    num_tokens = len(enc.encode(string))
    return num_tokens

In [ ]:
# Estimate number of tokens
num_tokens_from_string(system_message + user_message)

In [ ]:
# Run Code with error handling
attempts = 0
max_attempts = 3
success = False
while not success and attempts < max_attempts:
    try:
        results = client.chat.completions.create(
            model=deployment_4,
            messages=messages
        )
        success = True
    except TimeoutError as e:
        print("Timeout error occurred, trying again...")
        attempts += 1
        time.sleep(5)
if not success:
    print("Failed to get a response after several retries.")


In [ ]:
# Get results in JSON format
results_json = json.loads(results.choices[0].message.content)

In [ ]:
# Get Japanese Translation
results_json['Japanese']